In [1]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime

In [33]:
breakingnewsReq = requests.get("http://news.ltn.com.tw/list/BreakingNews")
breakingnewsSoup = BeautifulSoup(breakingnewsReq.text, "html.parser")

In [34]:
breakingnews = []
categoryDict = {"tab1":"焦點", "tab2":"政治","tab3":"社會","tab4":"生活","tab5":"國際","tab6":"言論","tab7":"財經","tab8":"體育","tab9":"地方","tab10":"娛樂","tab11":"消費","tab12":"副刊","tab13":"3C","tab14":"汽車","tab15":"iStyle"}
for titles, time, category in zip(breakingnewsSoup.select('.lipic .picword') , breakingnewsSoup.select('.lipic span') , breakingnewsSoup.select('.lipic span a')):
    url = titles.get('href')
    
    title = titles.text
    
    time = time.text
    time = time.replace("-" , "/")
    time = datetime.strptime(time, '%Y/%m/%d %H:%M')
    
    category = category.get('class')
    category = categoryDict[category[0]]
    
    #print url, title, time, category
    
    fbReq = requests.get('https://graph.facebook.com/?ids=' + url)
    fbData = fbReq.json()
    shareCount = fbData[url]['shares'] if fbData[url].has_key('shares') else 0
    likeCount = shareCount
    commentCount = fbData[url]['comments'] if fbData[url].has_key('comments') else 0
    
    detailReq = requests.get(url)
    detailSoup = BeautifulSoup(detailReq.text, "html.parser")
    
    content = []
    keyword = []
    
    contents = detailSoup.select('meta[property="og:description"]')
    for cont in contents:
        content.append(cont.get('content'))
    keywords = detailSoup.select('.con_keyword a')
    for key in keywords:
        keyword.append(key.text)
    img = detailSoup.select('meta[property="og:image"]')[0].get('content')

    breakingnews.append([title, category, keyword, shareCount, likeCount, commentCount, content, url, time, img])
    

In [3]:
for lists in LTNbreakingnews:
    [title, category, keywords, shareCount, likeCount, commentCount, content, url, time, img] = lists
    print "Title:" + title
    print "Category:" + category
    print "Keywords:",
    for key in keywords:
        print key,
    print
    print "ShareCount:",
    print shareCount
    print "LikeCount:", 
    print likeCount
    print "CommentCount:",
    print commentCount
    print "Content:",
    for c in content:
        print c
    print "Url:" + url
    print "Time:",
    print time
    print "Img:" + img

Title:日本商品展  新光三越高雄左營店登場
Category:生活
Keywords: <a href="/topic/%E6%96%B0%E5%85%89%E4%B8%89%E8%B6%8A">新光三越</a> <a href="/topic/%E6%97%A5%E6%9C%AC%E5%95%86%E5%93%81%E5%B1%95">日本商品展</a>
ShareCount: 0
LikeCount: 0
CommentCount: 0
Content: 每年都吸引大批人潮的日本商品展，今天起至20日在新光三越高雄左營店10樓登場。今年進入第5回的日本商品展，不僅再次出擊北海道、沖繩等當地名家，本回更擴大前往深受消費者喜愛的關西地區，親訪京都、大阪、神戶等地，嚴選當地名產、名店及文化工藝，網羅超過60個品牌跨海首登、邀請超過65位日籍職人現場實演，讓您不用搭飛機也能品嘗這些道地美食！
Url:http://news.ltn.com.tw/news/life/breakingnews/1534474
Time: 2015-12-09 16:18:00
Img:http://img.ltn.com.tw/Upload/liveNews/BigPic/600_1534474_1.jpg
Title:跟李敏鎬一起過耶誕節！歐爸12月來台必訪名店是...
Category:iStyle
Keywords:
ShareCount: 0
LikeCount: 0
CommentCount: 0
Content: 韓國男神李敏鎬12月15日即將現身台北，粉絲們除了從上個月開始拼運氣參加金莎抽獎活動，只求能當幸運粉絲跟敏鎬歐爸近距離互動之外，也一直打聽李敏鎬的班機時間及現身台北的確切時間，而這個祕密現在終於公開！
Url:http://istyle.ltn.com.tw/article/2579
Time: 2015-12-09 16:17:00
Img:http://img.ltn.com.tw/Upload/style/page/2015/12/09/151209-2579-0-pLwCZ.jpg
Title:基層拜票遇孕婦不適 王定宇助送醫
Category:政治
Keywords: <a href="/topic/2016%E7%B8%BD%E7%B5%B1%E5%

In [40]:
create_at = datetime.now();
col_LTN = MongoClient('192.168.99.100', 27017).iguana.LTN
col_LTN_count = MongoClient('192.168.99.100', 27017).iguana.LTN_count

In [16]:
for lists in hotnews:
    [title, category, keywords, shareCount, likeCount, commentCount, content, url, time, img] = lists
    content_dataObject = {
        "title": title, # 標題
        "category": category, # 分類
        "keywords" : keywords, #關鍵字
        "content": content, #內文
        "url" : url, # 網址
        "create_date" : time, # 發布時間
        "create_at" : create_at, #寫入時間
        "image" : img, # 圖片
        "source" : "Ettoday", # 來源
    }
    col_ettoday.update_one(
        {
           "title": title
        },
        {"$set": content_dataObject},
        upsert = True
    )
    news_id = col_ettoday.find_one({'title': title})['_id']
    count_dataObject = {
        "news_id" : news_id,
        "shareCount": shareCount, # 分享數
        "likeCount": likeCount, # 按讚數
        "commentCount": commentCount, #評論數
        "update_time" : create_at,
        "browserCount": None, # 瀏覽數
    }
    col_ettoday_count.update_one(
        {
        "shareCount": shareCount, # 分享數
        "likeCount": likeCount, # 按讚數
        "commentCount": commentCount, #評論數
        "update_time" : create_at
        },
        {"$set": count_dataObject} ,
        upsert = True
    )

In [39]:
for lists in breakingnews:
    [title, category, keywords, shareCount, likeCount, commentCount, content, url, time, img] = lists
    content_dataObject = {
        "title": title, # 標題
        "category": category, # 分類
        "keywords" : keywords, #關鍵字
        "content": content, #內文
        "url" : url, # 網址
        "create_date" : time, # 發布時間
        "create_at" : create_at, #寫入時間
        "image" : img, # 圖片
        "source" : "LTN", # 來源
    }
    col_LTN.update_one(
        {
           "title": title
        },
        {"$set": content_dataObject},
        upsert = True
    )
    news_id = col_LTN.find_one({'title': title})['_id']
    count_dataObject = {
        "news_id" : news_id,
        "shareCount": shareCount, # 分享數
        "likeCount": likeCount, # 按讚數
        "commentCount": commentCount, #評論數
        "update_time" : create_at,
        "browserCount": None, # 瀏覽數
    }
    col_LTN_count.update_one(
        {
        "news_id" : news_id,
        "shareCount": shareCount, # 分享數
        "likeCount": likeCount, # 按讚數
        "commentCount": commentCount, #評論數
        "update_time" : create_at
        },
        {"$set": count_dataObject} ,
        upsert = True
    )



In [4]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime

#LTN parse
LTNbreakingnewsReq = requests.get("http://news.ltn.com.tw/list/BreakingNews")
LTNbreakingnewsSoup = BeautifulSoup(LTNbreakingnewsReq.text, "html.parser")

LTNbreakingnews = []
categoryDict = {"tab1":"焦點", "tab2":"政治","tab3":"社會","tab4":"生活","tab5":"國際","tab6":"言論","tab7":"財經","tab8":"體育","tab9":"地方","tab10":"娛樂","tab11":"消費","tab12":"副刊","tab13":"3C","tab14":"汽車","tab15":"iStyle"}
for titles, time, category in zip(LTNbreakingnewsSoup.select('.lipic .picword') , LTNbreakingnewsSoup.select('.lipic span') , LTNbreakingnewsSoup.select('.lipic span a')):
    url = titles.get('href')

    title = titles.text

    time = time.text
    time = time.replace("-" , "/")
    time = datetime.strptime(time, '%Y/%m/%d %H:%M')

    category = category.get('class')
    category = categoryDict[category[0]]

    #print url, title, time, category

    fbReq = requests.get('https://graph.facebook.com/?ids=' + url)
    fbData = fbReq.json()
    shareCount = fbData[url]['shares'] if fbData[url].has_key('shares') else 0
    likeCount = shareCount
    commentCount = fbData[url]['comments'] if fbData[url].has_key('comments') else 0

    detailReq = requests.get(url)
    detailSoup = BeautifulSoup(detailReq.text, "html.parser")

    content = []
    keywords = []

    contents = detailSoup.select('meta[property="og:description"]')
    for cont in contents:
        content.append(cont.get('content'))
    keyword = detailSoup.select('.con_keyword a')
    for key in keyword:
        keywords.append(key.text)
    img = detailSoup.select('meta[property="og:image"]')[0].get('content')

    LTNbreakingnews.append([title, category, keywords, shareCount, likeCount, commentCount, content, url, time, img])


create_at = datetime.now();
col_LTN = MongoClient('192.168.99.100', 27017).iguana.LTN
col_LTN_count = MongoClient('192.168.99.100', 27017).iguana.LTN_count
for lists in LTNbreakingnews:
    [title, category, keywords, shareCount, likeCount, commentCount, content, url, time, img] = lists
    content_dataObject = {
        "title": title, # 標題
        "category": category, # 分類
        "keywords" : keywords, #關鍵字
        "content": content, #內文
        "url" : url, # 網址
        "create_date" : time, # 發布時間
        "create_at" : create_at, #寫入時間
        "image" : img, # 圖片
        "source" : "LTN", # 來源
    }
    col_LTN.update_one(
        {
           "title": title
        },
        {"$set": content_dataObject},
        upsert = True
    )
    news_id = col_LTN.find_one({'title': title})['_id']
    count_dataObject = {
        "news_id" : news_id,
        "shareCount": shareCount, # 分享數
        "likeCount": likeCount, # 按讚數
        "commentCount": commentCount, #評論數
        "update_time" : create_at,
        "browserCount": None, # 瀏覽數
    }
    col_LTN_count.update_one(
        {
            "news_id" : news_id,
            "shareCount": shareCount, # 分享數
            "likeCount": likeCount, # 按讚數
            "commentCount": commentCount, #評論數
            "update_time" : create_at
        },
        {"$set": count_dataObject} ,
        upsert = True
    )

